# Using `redo_lci`

Constructing an LCA object can take a while - it needs the following steps:

1. Resolve the dependency chain for the functional unit to determine which databases to load
1. Load the necessary processed arrays into memory
1. Index the arrays to the matrix row and column coordinates
1. LU factorize the technosphere matrix
1. Solve the specific functional unit linear algebra problem
1. Multiply the inventory matrix by the biosphere and (maybe) characterization matrices

If we want to do an LCA of another functional unit with the same databases, we can skip steps 1-4 and save a bunch of time. Let's see how much:

In [1]:
import brightway2 as bw

In [2]:
activities = [obj for obj, _ in zip(bw.Database("ecoinvent 3.6 cutoff"), range(100))]

## Naive approach

In [3]:
def create_object_each_time():
    for act in activities:
        lca = bw.LCA({act: 1})
        lca.lci()

In [4]:
%timeit create_object_each_time()

20 s ± 573 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Using `redo_lci`

`redo_lci` takes one argument: The new functional unit, in the same way as the LCA object creation does.

In [5]:
def reuse_LCA_object():
    lca = bw.LCA({activities[0]: 1})
    lca.lci()
    
    for act in activities:
        lca.redo_lci({act: 1})

In [6]:
%timeit reuse_LCA_object()

1.5 s ± 88.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Almost 20 times faster!

# Using `redo_lcia`

If you are doing LCIA, the `redo_lci` will only recalculate the *inventory*, not the *characterized_inventory*. Sometimes this is desirable; however, if you want the full calculation redone, use `redo_lcia`. Note that `redo_lcia` takes the same input argument as `redo_lci`.

In [7]:
def reuse_LCA_object_with_lcia():
    lca = bw.LCA({activities[0]: 1}, method=bw.methods.random())
    lca.lci()
    lca.lcia()
    
    for act in activities[:20]:
        lca.redo_lcia({act: 1})
        print(lca.score)

In [8]:
reuse_LCA_object_with_lcia()

3.0679529279153365e-05
1848.9036702689527
7.02603568874659e-07
3.918320009978089e-06
1.4327374994506628e-05
2.3350554390697607e-05
9.649130996149661e-08
1.6630029592881153e-05
0.00030497131499664576
0.5315208796902207
0.0
0.00018560085871578282
2.073410820648018e-06
0.0007577243235769673
0.8074576181017625
2.4631912611964484e-07
0.000167420139124996
-8.97699011348838e-06
-4.331149473802764e-06
420.45922051348424


# Multiple LCIA methods

You can also save some time if you have multiple impact categories by only building each characterization matrix once. The time savings here are not as dramatic, but can build up if you are doing a lot of calculations.

The key is to build each characterization matrix once, and store a copy of it in a list.

To switch LCIA methods, we use the `switch_method` method, which takes a method tuple as its input.

## A naive approach

In [9]:
methods = [bw.methods.random() for _ in range(10)]

In [10]:
def create_new_characterization_matrices_each_time():
    lca = bw.LCA({activities[0]: 1}, method=bw.methods.random())
    lca.lci()
    lca.lcia()
    
    for act in activities:
        lca.redo_lci({act: 1})
        for method in methods:
            lca.switch_method(method)
            score = (lca.characterization_matrix * lca.inventory).sum()

In [11]:
%timeit create_new_characterization_matrices_each_time()

4.08 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Only create the characterization matrices once

In [12]:
def reuse_characterization_matrices():
    lca = bw.LCA({activities[0]: 1}, method=bw.methods.random())
    lca.lci()
    lca.lcia()
    
    methods_list = []
    for method in methods:
        lca.switch_method(method)
        methods_list.append(lca.characterization_matrix.copy())
    
    for act in activities:
        lca.redo_lci({act: 1})
        for matrix in methods_list:
            score = (matrix * lca.inventory).sum()

In [13]:
%timeit reuse_characterization_matrices()

2.18 s ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Twice as fast, with almost no extra effort :)